<a href="https://colab.research.google.com/github/toufic12/LangChain-Chatbot-with-documents/blob/main/Rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

In [ ]:
!pip install langchain langchain_community langchain_openai langchain_core

In [3]:
from langchain_openai import ChatOpenAI
import os

In [4]:

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
Chat = ChatOpenAI(model="gpt-3.5-turbo-instruct", temperature=0.9)

In [5]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.chains import LLMChain
from langchain.llms import OpenAI

In [ ]:
!pip install rank_bm25 pypdf unstructured["pdf"] chromadb

In [7]:
from langchain.document_loaders import UnstructuredPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [8]:
embeddings = OpenAIEmbeddings()

In [9]:
path = "/content/Naked Forex - PDF Room.pdf"
loader = UnstructuredPDFLoader(path)
data = loader.load()


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [10]:
text_spliter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=50)

In [11]:
chunks= text_spliter.split_documents(data)

In [12]:
vectore_store= Chroma.from_documents(chunks, embeddings)

In [13]:
vectore_retriever = vectore_store.as_retriever()

In [14]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever

In [15]:
keyword_retriever = BM25Retriever.from_documents(chunks)

In [16]:
retriever = EnsembleRetriever(retrievers=[vectore_retriever, keyword_retriever], weghts=[0.5, 0.5])


In [17]:
template = """
You are an AI assitant that follows instructions extremly well.
Please be truthfull and give direct anwser

Content : {context}
</s>
<|user|>
{query}
</s>
<|assistant|>


"""

In [18]:
llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0.1)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [21]:
prompt = ChatPromptTemplate.from_template(template)

In [23]:
output_parsers = StrOutputParser()

In [25]:
chain = {"context":retriever, "query":RunnablePassthrough()} | prompt | llm | output_parsers

In [27]:
chain.invoke("The coclusion")

'The conclusion is that profits do not come from trading systems, but from traders themselves. The cycle of doom, where traders blame their losses on external factors, is a common trap that many traders fall into. It is important for traders to take responsibility for their own actions and decisions in order to find success in the market.'